In [1]:
Pkg.add("OffsetArrays") # -> Nos permitirá usar matrices que empiecen con índice 0 
using OffsetArrays

INFO: Cloning cache of OffsetArrays from https://github.com/JuliaArrays/OffsetArrays.jl.git
INFO: Installing OffsetArrays v0.3.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of OffsetArrays
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Precompiling module OffsetArrays.


# Programación dinámica

Veremos únicamente casos que se calificarían como programación dinámica determinista en horizonte finito (categoría en la que caen la mayoría de problemas de optimización combinatoria determinista).

Utilizaremos dos ejemplos (knapsack y lot sizing) que ya hemos visto y que luego intentaremos combinar/comparar con los métodos expuestos con anterioridad. 

No voy a repasar los fundamentos de la programación dinámica. Si alguien necesita un repaso más allá del nivel introductorio, recomiendo el libro de Denardo porque es uno de los más accesibles y baratos (E. Denardo, Dynamic Programming: Models and Applications).

# Uncapacitated Lot sizing

Este problema ya lo hemos visto en el diseño de cortes, así que hemos trasladado algunas funciones de la página anterior que nos resultarán útiles


In [2]:
function readULS(path::String)
    f = open(path,"r")
    T = parse(Int64,readline(f)) 
    c=map(x->parse(Int64,x),split(strip(readline(f))))
    h=map(x->parse(Int64,x),split(strip(readline(f))))
    K=map(x->parse(Int64,x),split(strip(readline(f))))
    d=map(x->parse(Int64,x),split(strip(readline(f))))
    close(f)
    T, c, h, K, d
end
function readSimpleInstance()
    T=5
    c=[10 10 10 10 10]
    h=[2 2 2 2 2]
    K=[75 75 75 75 75]
    d=[5 6 7 12 8]
    T,c,h,K,d
end

# T -> número de periodos
# c_t -> costo de comprar una unidad en el periodo t
# h_t -> costo de inventariar una unidad en el periodo t
# K_t -> costo de realizar un pedido en el periodo t
# d_t -> demanda en el periodo t

readSimpleInstance (generic function with 1 method)

In [3]:
run(`wget -O test.dat https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat`)
run(`cat test.dat`)
T, c, h, K, d = readULS("test.dat")

--2017-11-09 19:53:07--  https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1666 (1.6K) [text/plain]
Saving to: ‘test.dat’

     0K .                                                     100% 46.8M=0s

2017-11-09 19:53:08 (46.8 MB/s) - ‘test.dat’ saved [1666/1666]



200
1 4 1 2 4 4 1 4 1 3 1 4 2 1 4 4 1 1 2 4 2 2 4 2 2 1 4 1 4 1 3 4 1 1 4 1 3 4 1 3 1 2 4 3 4 3 1 4 1 2 4 2 3 1 2 2 4 2 1 3 2 2 4 1 1 2 2 1 4 1 3 1 2 3 2 1 4 3 2 2 2 2 4 4 3 1 3 4 2 3 2 1 1 1 1 1 1 4 1 3 3 3 1 1 2 3 3 3 3 2 3 2 2 2 4 1 3 4 1 3 2 2 4 4 1 1 2 3 4 4 4 3 1 1 2 2 3 3 1 4 1 1 4 1 4 1 2 1 4 3 3 2 2 3 4 2 3 4 1 4 2 4 4 3 3 1 4 4 2 4 1 4 2 1 3 3 4 3 3 4 4 1 4 4 4 3 1 1 3 2 2 1 1 1 4 3 3 3 3 2 
2 3 2 2 3 2 3 2 1 1 1 1 2 1 1 3 3 2 2 1 1 3 1 1 3 2 2 1 1 1 3 1 3 3 1 3 1 1 2 1 1 1 1 1 1 3 2 1 2 3 3 3 3 2 2 3 3 2 3 1 3 2 3 2 1 2 2 3 1 1 2 2 2 1 3 3 3 3 2 3 3 1 1 1 2 2 2 2 1 1 3 1 1 2 3 2 2 2 1 1 2 1 1 3 2 1 3 2 2 3 2 3 1 1 2 1 1 2 3 3 3 2 1 2 2 1 1 3 2 1 2 3 1 3 1 3 2 2 1 3 1 1 1 3 1 3 3 2 1 2 2 1 2 1 3 2 3 2 3 1 2 2 3 3 1 2 2 3 3 3 1 1 3 1 3 2 2 3 2 1 2 3 2 1 1 2 1 3 1 1 3 2 2 1 2 3 1 1 2 1 
10 8 6 6 10 8 9 5 10 10 5 7 8 10 5 7 6 5 8 6 8 10 9 6 9 10 5 5 10 6 9 8 10 10 10 10 7 5 9 7 10 8 9 10 8 7 9 7 10 10 8 9 5 9 5 8 8 10 5 9 5 5 7 9 9 9 9 7 7 7 10 5 5 7 9 9 6 7 7 7 9 6 8 8 8 10 9 6

(200,[1,4,1,2,4,4,1,4,1,3  …  2,1,1,1,4,3,3,3,3,2],[2,3,2,2,3,2,3,2,1,1  …  3,2,2,1,2,3,1,1,2,1],[10,8,6,6,10,8,9,5,10,10  …  7,9,6,6,7,7,7,5,7,10],[4,10,5,4,9,7,2,8,4,7  …  9,4,2,2,10,10,4,7,8,3])

## Uncapacitated Lot Sizing. Método 1. Estados asociados al nivel de inventario y recurrencia backward

Éste sería el método tradicional para explicar en clase. Sabemos que en el periodo T+1 el costo es 0 y la última decisión corresponde al periodo T. 

Los estados $s$ representan el nivel de inventario

Las Transiciones $x$ indican el número  unidades que compramos en este periodo.

La función de recurrencia es la siguiente:

\begin{equation}
f^N(s)=\min_{\max\{0,d_{T+1-N}-s \} \leq x\leq D} \left\{ f^{N+1}\left(s+x-d_{T+1-N}\right) + K_{T+1-N}  sgn(x)  + h_{T+1-N}  (s+x-d_{T+1-N}) + c_{T+1-N}   x \right\}
\end{equation}

La estructura del cálculo se inicia cuando quedan N=1 decisiones por tomar. En tal caso sabemos que compramos hasta la demanda si tenemos menos inventario que demanda o 0 si tenemos más inventario que demanda (corresponde a las líneas 5 a 12)

A partir de ahí miramos cuando quedan 2, 3, ..., N decisiones por tomar. Calculamos cada estado $st$ mirando todas sus alternativas como tenemos que escoger el máximo de las opciones, primero seleccionamos una (que depende de si $s\geq d_{T+1-N}$) y luego evaluamos todas las demás para ver si alguna mejora (filas 13 a 43).

Guardamos dos tablas. La primera guarda los costos, la segunda guarda la traza para la reconstrucción.

Finalmente usamos la tabla de traza para reconstruir la solución.

In [ ]:
function DP_m1(T,c,h,K,d)
    D=sum(d)
    tableCosts = OffsetArray(Int64, 1:T, 0:D)
    tablePath = OffsetArray(Int64, 1:T, 0:D)
    for i=0:d[T]-1
        tableCosts[1,i]=K[T]+c[T]*(d[T]-i) #hacer un pedido y comprar lo que falta hasta d[T]
        tablePath[1,i]=0
    end
    for i=d[T]:D
        tableCosts[1,i]=h[T]*(i-d[T]) #no pedir y pagar inventario
        tablePath[1,i]=i-d[T] 
    end
    for t=2:T #para cada etapa
        period=T-t+1
        for st=0:D
            #hay que diferenciar dos casos
            if st<d[period] #es obligatorio comprar
                #iniciamos con la opción de comprar justo para la semana
                tableCosts[t,st]=K[period]+c[period]*(d[period]-st)+tableCosts[t-1,0]
                tablePath[t,st]=0
                #resto de opciones consisten en comprar desde d[period]-st+1 hasta D-st
                for tr=d[period]-st+1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost < tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            else
                #inicializamos con la opción de no comprar
                tableCosts[t,st]=h[period]*(st-d[period])+tableCosts[t-1,st-d[period]]
                tablePath[t,st]=st-d[period]
                #resto de opciones consisten en comprar desde 1 hasta D-st
                for tr=1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost<tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            end
        end
    end
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    t=T
    currentState=0
    while t>=1
        if tablePath[t,currentState] > currentState-d[T+1-t]
            ySol[T+1-t]=1
            xSol[T+1-t]=tablePath[t,currentState]+d[T+1-t]
        end
        currentState=tablePath[t,currentState]
        t -= 1
    end
    return ySol,xSol,tableCosts[T,0]
end

In [ ]:
#T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m1(T,c,h,K,d)
println(ySol)
println(xSol)
println(totalCost)

## Uncapacitated Lot Sizing. Método 2. Estados asociados al nivel de inventario y recurrencia forward

Aunque existe una recurrencia en sentido forward mimética a la mostrada en el método 1, aquí se opta por organizar los cálculos de una manera distinta (para mostrar la variedad de implementaciones).

Si en el caso anterior se evaluaban todas las opciones de llegar a un estado y se seleccionaba la mejor, ahora se evaluarán todas las opciones que pueden generarse desde un estado y se mantendrá siempre la mejor.

Esto se hace para mostrar varias alternativas de resolución de las recurrencias (no siempre es trivial ver cuál es mejor y además no siempre conviene resolver las recurrencias como se haría en papel).

La idea es la siguiente: 

* En el periodo 1 compramos un número de unidades (entre d[1] y D) 
* En el periodo 2 podemos encontrarnos en diferentes niveles de inventario. En cualquiera de ellos podemos optar por comprar cantidades diferentes. Compremos lo que compremos estamos generando un posible estado del periodo 3 y podemos llegar a ese valor de muchas y distintas maneras (por ejemplo por comprar x en el periodo 1 e y en el periodo 2, o y en el periodo 1 y x en el periodo 2; en cualquiera de los dos casos empezaremos el periodo 3 con la misma cantidad de inventario).

La clave ahora es que cada vez que generemos un estado de una etapa, verifiquemos si el nuevo método para alcanzar ese estado es mejor o peor y actualicemos consecuentemente los costes y la traza.

Finalmente, en el periodo T resulta trivial determinar cuál es la mejor opción (siempre procede de tener el inventario justo suficiente o de comprar la demanda justa de ese periodo).

El código se estructura como sigue:

Las transiciones desde el único estado de la etapa 1 se resuelve en las líneas 5 a 8. 

De las líneas 9 a 28 se evalúa la recurrencia, primero generando transiciones desde el estado asociado a 0 inventarios (líneas 11 a 14) y luego desde el resto de estados (líneas 15 a 27)

Para ahorrar cálculos se eliminan transiciones que sabemos que nunca se podrían alcanzar (más inventario de la demanda que resta hasta el final del periodo).

In [ ]:
function DP_m2(T,c,h,K,d)
    tableCosts = OffsetArray(Int64, 2:T, 0:sum(d))
    tablePath = OffsetArray(Int64, 2:T, 0:sum(d))
    #etapa 1
    for tr=d[1]:sum(d)
        tableCosts[2,tr-d[1]]=K[1]+c[1]*tr+h[1]*(tr-d[1])
        tablePath[2,tr-d[1]]=0
    end
    for t=2:T-1
        #empiezo con estado stock=0 para inicializar tableCosts[t+1,:] y tablePath[t+1,:]
        for tr=0:sum(d[t+1:T]) #se inicializa sumando costos hasta ese punto + costo de la compra en ese periodo
            tableCosts[t+1,tr]=tableCosts[t,0]+K[t]+c[t]*(d[t]+tr)+h[t]*tr
            tablePath[t+1,tr]=0
        end
        for s=1:sum(d[t:T]) #estados en los que me puedo encontrar
            for tr=max(0,d[t]-s):max(0,sum(d[t:T])-s) #aqui una transición indica cuántas unidades se van a comprar
                if tr==0
                    cost=tableCosts[t,s]+h[t]*(s+tr-d[t])
                else
                    cost=tableCosts[t,s]+K[t]+c[t]*tr+h[t]*(s+tr-d[t])
                end
                if cost<tableCosts[t+1,s+tr-d[t]]
                    tableCosts[t+1,s+tr-d[t]]=cost
                    tablePath[t+1,s+tr-d[t]]=s
                end
            end
        end
    end
   
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    totalCost=0
    currentState=0
    #el óptimo corresponde a tener exactamente 0 o d[T] unidades en inventario
    if (tableCosts[T,0]+K[T]+c[T]*d[T]) < tableCosts[T,d[T]] 
        #interesa comprar en el ultimo periodo
        ySol[T]=1
        totalCost=tableCosts[T,0]+K[T]+c[t]*d[T]
        currentState=tablePath[T,0]
    else
        #interesa comprar antes del último periodo
        totalCost=tableCosts[T,d[T]]
        currentState=tablePath[T,d[T]]
    end
    t=T-1
    while t>=2
        if currentState==0
            ySol[t]=1
        end
        currentState=tablePath[t,currentState]
        t -= 1
    end
    ySol[1]=1
    for t=1:T
        if ySol[t]==1
            xSol[t] += d[t]
            for tt=t+1:T
                if ySol[tt]==1
                    break
                end
                xSol[t] += d[tt]
            end
        end
    end
    return ySol,xSol,totalCost
end

In [ ]:
#T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m2(T,c,h,K,d)
println("ySol: ",ySol)
println("xSol: ",xSol)
println("Cost: ",totalCost)

## Uncapacitated Lot Sizing. Método 3. Estados asociados al nivel de inventario, recurrencia forward y transiciones limitadas 

Partiendo de las ideas ya vistas en el método 2, uno puede percibir que el conjunto de estados "útiles" es mucho más pequeño que el conjunto de estados "posibles". Por tanto, podríamos aprovechar que la solución es "sparse". 

Para ello vamos a necesitar estructuras de datos algo más complejas y vamos a proceder con la recurrencia forward por comodidad.

Usaremos dos listas, una con los estados de la etapa actual y otro con los estados de la etapa siguiente. Además mantendremos un diccionario con las transiciones generadas (esto será útil para reconstruir la solución)

In [ ]:
#usaremos un "trick" en stateSpace, un estado será etapa*10000+estado

function DP_m3(T,c,h,K,d)
    cte=10000
    state = 0
    stateSpace = Dict{Int64,Int64}() #todos los estados para hacer reconstruccion
    currentStage = Dict{Int64,Int64}() #etapa actual
    nextStage = Dict{Int64,Int64}() #etapa siguiente
    
    t=1
    currentStage[0]=0
    stateSpace[1*cte]=0
    for t=1:T
        for k in keys(currentStage)
            if k==0 #si no tengo inventario, decisión de compra por periodos consecutivos
                compro=0
                costo = K[t] 
                for tt=t:T
                    compro += d[tt]
                    costo += c[t]*d[tt]
                    nextState=compro-d[t]
                    if haskey(nextStage,nextState)
                        if nextStage[nextState]> (costo + h[t]*(compro-d[t]) + currentStage[k]) #mejora costo
                            nextStage[nextState]= costo + h[t]*(compro-d[t]) + currentStage[k]
                            stateSpace[(t+1)*cte+nextState]=t*cte+k
                        end
                    else
                        nextStage[nextState]=costo + h[t]*(compro-d[t]) + currentStage[k]
                        stateSpace[(t+1)*cte+nextState]=t*cte+k
                    end
                end
            else #si tengo inventario la decisión es no comprar
                nextState=k-d[t]
                if haskey(nextStage,nextState)
                    if nextStage[nextState] > ( currentStage[k] + h[t]*(k-d[t]) )
                        nextStage[nextState] = currentStage[k] + h[t]*(k-d[t])
                        stateSpace[(t+1)*cte+nextState]=t*cte+k
                    end                        
                else
                    nextStage[nextState] = currentStage[k] + h[t]*(k-d[t])
                    stateSpace[(t+1)*cte+nextState]=t*cte+k
                end
            end
            delete!(currentStage,k) #lo elimino
        end
        nextStage,currentStage = currentStage,nextStage
    end
    st=(T+1)*cte
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    st=stateSpace[st]
    while st!=0
        stage=trunc(Int, st/cte+0.00001)
        remaining=st%cte
        if remaining==0 && stage>0
            ySol[stage]=1
        end
        st=stateSpace[st]
    end
    for t=1:T
        if ySol[t]==1
            xSol[t] += d[t]
            for tt=t+1:T
                if ySol[tt]==1
                    break
                end
                xSol[t] += d[tt]
            end
        end
    end
    #hay un único estado en currentStage que es el estado con 0 stock (sol. final)
    return ySol,xSol,currentStage[0]
end

In [ ]:
#T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m3(T,c,h,K,d)
println("ySol: ",ySol)
println("xSol: ",xSol)
println("Cost: ",totalCost)

## Uncapacitated Lot Sizing. Método 4. Estados asociados a los periodos y recurrencia forward

En este, el conjunto de estados es todavía mucho menor y podemos operar sobre los periodos en que se realizará una compra.

En tal caso el problema conviene tratarlo como un problema de caminos (un caso particular de un problema de programación dinámica).

In [ ]:
function DP_m4(T,c,h,K,d)
    ∞=10000000
    costs=fill(∞,T+1)
    path=zeros(Int64,T+1)
    costs[1]=0
    for t in 1:T
        costAc=costs[t]+K[t]+c[t]*d[t]
        hAc=h[t]
        for tt=t+1:T+1
            if costAc<costs[tt]
                costs[tt]=costAc
                path[tt]=t
            end
            if tt<=T
                costAc +=c[t]*d[tt]+hAc*d[tt]
                hAc += h[tt]
            end
        end
    end
    
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    st=T+1
    while st!=1
        st=path[st]
        ySol[st]=1
    end
    for t=1:T
        if ySol[t]==1
            xSol[t] += d[t]
            for tt=t+1:T
                if ySol[tt]==1
                    break
                end
                xSol[t] += d[tt]
            end
        end
    end
    return ySol,xSol,costs[T+1]
end

In [ ]:
#T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m4(T,c,h,K,d)
println("ySol: ",ySol)
println("xSol: ",xSol)
println("Cost: ",totalCost)

### Ejercicio Extra

Hay dos cosas en las implementaciones 1 y 2 que son discutibles (sobretodo a nivel de tiempo de cómputo). 

La primera es el uso del paquete "OffsetArrays" que es más lento que una implementación directa. 

La segunda es el usar como primer índice el tiempo y como segundo el estado. Podría ser más rápido usar un orden inverso. 

¿Podría dar una respuesta teórica y validarla empíricamente?

Otra opción es plantearse cómo resolver el problema de forma más eficiente (estos algoritmos son $n^2$ pero podríamos resolver el problema en $n \log(n)$ (ver 10.1287/opre.41.3.549, 10.1287/mnsc.37.8.909 o 10.1287/opre.40.1.S145)